In [ ]:
!pip install kfp
!pip install google-cloud-pipeline-components

In [ ]:
import json

from kfp import dsl
from kfp.v2 import compiler
from kfp.v2.dsl import component
from datetime import datetime
from kfp.components import create_component_from_func
import typing

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

In [ ]:
## Light weight component to create a Tabular DS
@component(
    base_image="python:3.9-slim",
    packages_to_install=["google-cloud", "google-cloud-aiplatform", "typing"],
)
def create_ds(bq_table: str) -> typing.NamedTuple('Outputs', [('bq_ds', str)]):
    from google.cloud import aiplatform
    from datetime import datetime
    import typing
    
    #Create DS
    TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
    dataset = aiplatform.TabularDataset.create(
        display_name=f'{TIMESTAMP}-dataset',
        bq_source=bq_table,
        location="us-central1",
        sync=True
    )
    
    bq_ds_uri = dataset.to_dict()["metadata"]["inputConfig"]["bigquerySource"]["uri"].split("bq://")[1].split(".")[1]
    
    return (bq_ds_uri, )

## Light weight component to delete a BQ DS
@component(
    base_image="python:3.9-slim",
    packages_to_install=["google-cloud", "google-cloud-bigquery"],
)
def delete_ds(bq_ds: str):
    from google.cloud import bigquery

    client = bigquery.Client()
    client.delete_dataset(bq_ds, delete_contents=True, not_found_ok=True)

In [ ]:
@dsl.pipeline(
    name="control-{}".format(TIMESTAMP),
    pipeline_root="gs://gcs-hasanrafiq-test-331814/pipeline_root",
)
def pipeline(bq_table: str):
    #Create Tabular DS
    dataset = create_ds(bq_table)
    
    #Delete DS
    delete_ds(str(dataset.outputs["bq_ds"]))

In [ ]:
from kfp.v2 import compiler

compiler.Compiler().compile(
    pipeline_func=pipeline, package_path="control_pipeline.json".replace(" ", "_")
)

In [ ]:
DISPLAY_NAME = "control_" + TIMESTAMP

job = aip.PipelineJob(
    display_name=DISPLAY_NAME,
    template_path="control_pipeline.json".replace(" ", "_"),
    parameter_values={"bq_table": "bq://hasanrafiq-test-331814.dummy_test2.address_copy"}
)

job.run()